In [1]:
!pip install -q groq

In [2]:
from time import time
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import display, Markdown
import os
import time
from groq import Groq

In [3]:
client = Groq(
    api_key='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'
)

In [4]:
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [5]:
import json
# Specify the path to the JSON file
file_path = "/kaggle/input/segresnet-result-json/1_ms_lesion_areas_.json"

# Load the JSON file
with open(file_path, "r") as file:
    data = json.load(file)

# Now you can work with the loaded JSON data
print(data)

SegResNetResults = "/kaggle/input/segresnet-result-json/SegResNet_results.json"
with open(SegResNetResults, "r") as file:
    res = json.load(file)
print(res)

{'Frontal-to-Occipital (GapMap) left': {'Percentage of Lesion': 26.88, 'Percentage of Region': 6.04}, 'Frontal-to-Occipital (GapMap) right': {'Percentage of Lesion': 16.82, 'Percentage of Region': 4.08}, 'Area hOc1 (V1, 17, CalcS) left': {'Percentage of Lesion': 8.88, 'Percentage of Region': 7.31}, 'Area hOc6 (POS) left': {'Percentage of Lesion': 4.23, 'Percentage of Region': 51.55}, 'Frontal-to-Temporal-II (GapMap) left': {'Percentage of Lesion': 3.1, 'Percentage of Region': 2.86}}
{'nDSC': 71.75595231995298, 'nDSC std': 8.696038792793306, 'Lesion F1 score': 35.110906675494164, 'Lesion F1 score std': 14.263135368144157, 'nDSC R-AUC': 8.74497076414209, 'nDSC R-AUC std': 8.162803573241632}


In [6]:
PROMPT1 = """
Using the provided JSON data from a brain MRI study of the lesion segmentation computed by the AI model, please generate a comprehensive report detailing the segmentation results of MS lesions. Describe the lesions' presence in various regions according to the Julich-Brain Atlas noting the percentage  of the lesion area within each region and the impact on the region in order to enhance the explainability of the AI lesion segmentation model to support clinical decision-making. 
Here is the json:
"""
PROMPT1 = PROMPT1 + json.dumps(data) + json.dumps(res)+"\n."

In [ ]:
PROMPT2="""
Using the provided JSON data from a brain MRI study of the lesion segmentation computed by the AI model, please generate a comprehensive report detailing the segmentation results of MS lesions. 
Describe the lesions' presence in various regions according to the Julich-Brain Atlas noting the percentage of the lesion area within each region and the impact on the region in order to enhance the explainability of the AI lesion segmentation model to support clinical decision-making.

The report should be structured as follows:
1. **Summary of Lesion Segmentation**: Provide an overview of the lesions' distribution across various regions of the brain according to the Julich-Brain Atlas. Discuss the semantic segmentation of the lesions and their implications.
2. **Detailed Regional Impact**: For each region where the lesion is present, describe the percentage of the lesion within the region and the percentage of the region affected by the lesion. Discuss the known roles and functions of these regions in the brain and how the presence of the lesion might impact these functions.
3. **Semantic Segmentation**: Discuss the different aspects of the lesions identified by the semantic segmentation and their potential clinical implications.

Here is the json:
"""

PROMPT2 = PROMPT2 + json.dumps(data) + json.dumps(res)

In [7]:
def generate_answer_groq(question):
    prompt = f"""
    You are an AI medical assistant designed to answer questions.
    Please restrict your answer to the exact question asked.
    Question: {question}
    Answer:
    """

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="mixtral-8x7b-32768",
    )

    return chat_completion.choices[0].message.content

In [8]:
response = generate_answer_groq(PROMPT1)
display(Markdown(colorize_text(response)))
with open("/kaggle/working/mixtral-8x7b_response_PROMPT1.MD", "w") as file:
    file.write(response)

Based on the provided JSON data, the AI model has segmented multiple sclerosis (MS) lesions in various regions of the brain. Here's a detailed report:

1. Frontal-to-Occipital (GapMap) left:
   - Percentage of Lesion: 26.88%
   - Percentage of Region: 6.04%
   The lesion occupies a significant portion of this region, which could impact the overall connectivity and function of the left hemisphere.

2. Frontal-to-Occipital (GapMap) right:
   - Percentage of Lesion: 16.82%
   - Percentage of Region: 4.08%
   The lesion in the right hemisphere also represents a substantial portion of this region, potentially affecting the communication and functions associated with this area.

3. Area hOc1 (V1, 17, CalcS) left:
   - Percentage of Lesion: 8.88%
   - Percentage of Region: 7.31%
   The presence of a lesion in the primary visual cortex could lead to visual deficits in the left visual field of the patient's vision.

4. Area hOc6 (POS) left:
   - Percentage of Lesion: 4.23%
   - Percentage of Region: 51.55%
   A relatively smaller lesion is present in the posterior parietal area, which might still affect the spatial awareness and processing of the left hemisphere.

5. Frontal-to-Temporal-II (GapMap) left:
   - Percentage of Lesion: 3.1%
   - Percentage of Region: 2.86%
   A minor lesion is identified in this region, which could contribute to language or memory-related symptoms.

Regarding the AI model's performance, key evaluation metrics include:
- nDSC (Normalized Dice Similarity Coefficient): 71.76%
  This indicates a good overlap between the AI-segmented lesions and the ground truth, suggesting reliable performance.
- Lesion F1 score: 35.11%
  This score is relatively low, which implies that there is room for improvement, mainly because the AI model missed many smaller lesions in the brain.

In conclusion, the AI model accurately segments major MS lesions across several brain regions while potentially missing some smaller lesions. The results highlight the importance of AI in clinical decision-making and potential applications for MS lesion segmentation.